# **Detecção de Fraudes em Transações Financeiras**

Luiz Henrique Rigo Faccio | CCR de `Inteligência Artifical`

*Ciência da Computação - Universidade Federal Da Fronteira Sul*

Dataset disponível em: [https://www.kaggle.com/datasets/aryan208/financial-transactions-dataset-for-fraud-detection](https://www.kaggle.com/datasets/aryan208/financial-transactions-dataset-for-fraud-detection)

## **Importando bibliotecas e o dataset**

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import datetime as dt
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
path = "archive/financial_fraud_detection_dataset.csv"
dataSet = pd.read_csv(path)

## **Vizualizando as informações**

In [3]:
def numeric_columns(dataSet):
    return dataSet.select_dtypes(include=['int64', 'float64']).columns
def categorical_columns(dataSet): 
    return dataSet.select_dtypes(include=['object']).columns

def analyze_data(dataSet):
    info = pd.DataFrame({"Tipos":dataSet.dtypes, "Valores únicos": dataSet.nunique(), "Valores Nulos": dataSet.isnull().sum()})
    
    print("Dimensão do dataset: ", dataSet.shape)
    display(info)

    print("Informações contínuas:")
    display(dataSet[numeric_columns(dataSet)].describe())

    if (categorical_columns(dataSet).size > 0):
        print("Informações categóricas:")
        display(dataSet[categorical_columns(dataSet)].describe())

    print("Amostra do dataset:")
    display(dataSet.sample(5))

In [4]:
#analyze_data(dataSet)

## **Tratando os dados**

Algumas informações como IDs, tipo de fraude, números de contas e números de dispositivos são inúteis nesta situação

time_since_last_transaction tem muitos valores vazios

In [5]:
useLess = ["transaction_id", "sender_account", "receiver_account", "ip_address", "device_hash", "fraud_type", "time_since_last_transaction"]

dataSet = dataSet.drop(columns=useLess)

Para não descartar os horários das transações, eles serão agrupados em madrugada, manha, tarde e noite.

In [6]:
def categorize_timestamp(timestamps : pd.Series):
    """Função para categorizar timestap em períodos: manhã, tarde, noite e madrugada

    Args:
        timestamps (pd.Series): Coluna de timestamp do DataSet
    
    Returns:
        periodos (pd.Series): Coluna de timestamps já categorizada
    """
    
    def get_period(hour):
        if 6 <= hour < 9:
            return "manha_1"
        if 9 <= hour < 12:
            return "manha_2"
        elif 12 <= hour < 15:
            return "tarde_1"
        elif 15 <= hour < 18:
            return "tarde_2"
        elif 18 <= hour < 21:
            return "noite_1"
        elif 18 <= hour < 21:
            return "noite_2"
        elif 21 <= hour < 23:
            return "tarde_2"
        elif 23 <= hour < 2:
            return "tarde_2"
        elif 2 <= hour < 5:
            return "madrugada_1"
        else:
            return "madrugada_2"
        
    periodos = timestamps.apply(lambda x: get_period(dt.datetime.fromisoformat(x).hour))
    return periodos
    

In [7]:
dataSet["timestamp"] = categorize_timestamp(dataSet["timestamp"])

Como não existem mais dados faltantes, não será necessário imputá-los

As informações categóricas serão tratadas com o uso de OneHotEncoding e os dados numéricos serão escalados com o uso de StandartScaler

Os valores de y (is_fraud) serão transformados em inteiros

In [8]:
sacaler = StandardScaler()
dataSet[numeric_columns(dataSet)] = sacaler.fit_transform(dataSet[numeric_columns(dataSet)])

dataSet["is_fraud"] = dataSet["is_fraud"].map(lambda x: 1 if x == True else 0)

In [9]:
encoder = OneHotEncoder()
encoder.fit(dataSet[categorical_columns(dataSet)])
encoded_data = encoder.transform(dataSet[categorical_columns(dataSet)])
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(categorical_columns(dataSet)))
dataSet = dataSet.drop(columns=categorical_columns(dataSet))
dataSet = pd.concat([dataSet, encoded_df], axis=1)

In [10]:
analyze_data(dataSet)

Dimensão do dataset:  (5000000, 40)


,Tipos,Valores únicos,Valores Nulos
amount,float64,217069,0
is_fraud,int64,2,0
spending_deviation_score,float64,917,0
velocity_score,float64,20,0
geo_anomaly_score,float64,101,0
timestamp_madrugada_1,float64,2,0
timestamp_madrugada_2,float64,2,0
timestamp_manha_1,float64,2,0
timestamp_manha_2,float64,2,0
timestamp_noite_1,float64,2,0


Informações contínuas:


,amount,is_fraud,spending_deviation_score,velocity_score,geo_anomaly_score,timestamp_madrugada_1,timestamp_madrugada_2,timestamp_manha_1,timestamp_manha_2,timestamp_noite_1,...,location_Tokyo,location_Toronto,device_used_atm,device_used_mobile,device_used_pos,device_used_web,payment_channel_ACH,payment_channel_UPI,payment_channel_card,payment_channel_wire_transfer
count,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,...,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06
mean,7.016183e-17,3.591060e-02,1.947598e-17,-6.677539e-17,-8.860610e-16,1.250936e-01,1.666556e-01,1.250348e-01,1.251578e-01,1.250794e-01,...,1.251988e-01,1.248698e-01,2.499280e-01,2.502262e-01,2.498316e-01,2.500142e-01,2.500482e-01,2.497694e-01,2.499386e-01,2.502438e-01
std,1.000000e+00,1.860673e-01,1.000000e+00,1.000000e+00,1.000000e+00,3.308250e-01,3.726681e-01,3.307584e-01,3.308978e-01,3.308090e-01,...,3.309442e-01,3.305713e-01,4.329712e-01,4.331433e-01,4.329155e-01,4.330209e-01,4.330406e-01,4.328795e-01,4.329773e-01,4.331534e-01
min,-7.637771e-01,0.000000e+00,-5.255371e+00,-1.647578e+00,-1.732394e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,-7.072584e-01,0.000000e+00,-6.790640e-01,-9.539571e-01,-8.662475e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,-4.687139e-01,0.000000e+00,3.878031e-04,8.647374e-02,-1.013599e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.084602e-01,0.000000e+00,6.698476e-01,9.534994e-01,8.660447e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,6.727840e+00,1.000000e+00,5.016341e+00,1.647120e+00,1.732191e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Amostra do dataset:


,amount,is_fraud,spending_deviation_score,velocity_score,geo_anomaly_score,timestamp_madrugada_1,timestamp_madrugada_2,timestamp_manha_1,timestamp_manha_2,timestamp_noite_1,...,location_Tokyo,location_Toronto,device_used_atm,device_used_mobile,device_used_pos,device_used_web,payment_channel_ACH,payment_channel_UPI,payment_channel_card,payment_channel_wire_transfer
4275455,0.852708,0,0.210218,0.953499,0.173128,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2206061,-0.750116,0,-0.788975,-0.260337,1.489670,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3705689,0.483230,0,1.659049,-1.127362,-0.311914,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3903299,-0.260642,0,-0.649088,-0.607147,1.073920,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1226568,0.284244,0,-0.129507,-1.647578,-0.311914,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## **Treinando Modelos**

Separando os dados de treino e de teste

Realizando ao treinamento dos modelos

In [11]:
X_train, X_test, y_train, y_test = train_test_split(dataSet.drop(columns=["is_fraud"]), dataSet["is_fraud"], test_size=0.2, random_state=42)

# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(
    max_depth=None,
    min_samples_split=10,
    min_samples_leaf=5,
)
dt_classifier.fit(X_train, y_train)
print("Decision Tree Classifier Score: ", dt_classifier.score(X_test, y_test))

# Neural Network Classifier
nn_classifier = MLPClassifier(
    hidden_layer_sizes=(40, 100, 150, 50, 2),
    activation='relu',
    solver='adam',
    max_iter=300,
    learning_rate_init=0.001,
)
nn_classifier.fit(X_train, y_train)
print("Neural Network Classifier Score: ", nn_classifier.score(X_test, y_test))

Decision Tree Classifier Score:  0.956187
Neural Network Classifier Score:  0.964179
